In [5]:
from DBcm import UseDatabase
import os
from datetime import datetime, timedelta

dataset_path = r""

def ingest(dataset):
    config = {
        "host": "localhost",
        "database": "SwimclubDB",
        "user": "swimuser",
        "password": "swimuserpasswd"
    }

    files = get_files(dataset)
    if ".DS_Store" in files:
        files.remove(".DS_Store")


    # start db connection
    with UseDatabase(config) as cursor:
        swimmer_data = set()
        events = set()
        
        for f in files:
            f = f.removesuffix(".txt")
            name, age, distance, stroke = f.split("-")
            distance = distance.removesuffix("m")
            distance = int(distance)
            
            if (distance, stroke) not in events:
                events.add((distance, stroke))
                _SQL = f"""insert ignore into events (distance, stroke) values ({distance}, '{stroke}');"""
                cursor.execute(_SQL)
            
            if (name, age) not in swimmer_data:
                swimmer_data.add((name, age))
                _SQL = f"""insert ignore into swimmers (name, age) values ('{name}', {age});"""
                cursor.execute(_SQL)
            
            _SQL = f"""select id from swimmers where name = '{name}' and age = {int(age)};"""
            cursor.execute(_SQL)
            swimmer_id = cursor.fetchall()[0][0]
            
            _SQL = f"""select id from events where distance = {distance} and stroke = '{stroke}';"""
            cursor.execute(_SQL)
            event_id = cursor.fetchall()[0][0]
    
            lines = read_file(dataset, f"{f}.txt")
            for line in lines:
                times = line.split(",")
                for time in times:
                    time = time.strip()
                    """
                    Note: The following conditional statement is only necessary to ensure that there is a difference in the times that the data is added for each dataset. This is done for demonstration purposes only and is not an intended feature of the application.
                    """
                    if dataset == "dataset_1":
                        ts = datetime.now() - timedelta(days=1)
                        _SQL = f"""insert ignore into times (swimmer_id, event_id, time, ts) VALUES ({swimmer_id}, {event_id}, '{time}', '{ts}');"""
                    else:   
                        _SQL = f"""insert ignore into times (swimmer_id, event_id, time) VALUES ({swimmer_id}, {event_id}, '{time}');"""
                    cursor.execute(_SQL)
                    

def get_files(dataset):
    path = rf"{dataset_path}{dataset}"
    return os.listdir(path)


def read_file(dataset, filename):
    path = rf"{dataset_path}{dataset}/{filename}"
    with open(path) as f:
        return f.readlines()
    
ingest("dataset_1")
ingest("dataset_2") 